<a href="https://colab.research.google.com/github/bhagesh-codebeast/hackathon/blob/dev/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install azure-ai-formrecognizer==3.2.0
! pip install azure-core
! python -m pip install azure-ai-vision
! wget -O - https://www.openssl.org/source/openssl-1.1.1u.tar.gz | tar zxf -
! cd openssl-1.1.1u
! ./config --prefix=/usr/local
! make -j $(nproc)
! sudo make install_sw install_ssldirs
! sudo ldconfig -v
! export SSL_CERT_DIR=/etc/ssl/certs
! cd -
! sudo apt-get install build-essential libssl-dev wget

In [ ]:
import os
import azure.ai.vision as sdk
from statistics import mean
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

In [ ]:
# use your `key` and `endpoint` environment variables
key = ""
endpoint = "https://robocoptest.cognitiveservices.azure.com/"
formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/read.png"

In [ ]:
# OCR Function
class do_ocr:
  def __init__(self, formUrl, model='prebuilt-read'):
    self.formUrl = formUrl
    self.model = model
  def analyze_read(self):
      document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))
      poller = document_analysis_client.begin_analyze_document_from_url(self.model, self.formUrl)
      result = poller.result()
      dict_page = {}
      word_confidence = []
      for idx, style in enumerate(result.styles):
          print("Document contains {} content".format("handwritten" if style.is_handwritten else "no handwritten"))
      for page in result.pages:
          dict_page['page_number'] = {page.page_number}
          for word in page.words:
            word_confidence.append({word.content:word.confidence})
          dict_page['content'] = word_confidence
      return result.content, dict_page
  def preprocess_results(self):
      content, predictions = self.analyze_read()
      average_score = mean(value for dicts in predictions.get('content', []) for value in dicts.values())
      final_dict = {key:value for dicts in predictions.get('content', []) for key, value in dicts.items() if value < average_score}
      return content, average_score, final_dict

if __name__ == "__main__":
    instance = do_ocr(formUrl).preprocess_results()
    content, average_score, final_dict = instance

In [ ]:
# Output

content
average_score
final_dict

In [ ]:
VISION_KEY=""
VISION_ENDPOINT="https://objectclassify.cognitiveservices.azure.com/"
url="https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png"
service_options = sdk.VisionServiceOptions(VISION_ENDPOINT,VISION_KEY)

In [ ]:
# OBject Classification

class do_classification:
  def __init__(self,url,service_options,language='en'):
    self.url = url
    self.language = language
    self.service_options = service_options
  def do_analysis(self):
    vision_source = sdk.VisionSource(url=self.url)
    analysis_options = sdk.ImageAnalysisOptions()
    analysis_options.features = (sdk.ImageAnalysisFeature.CAPTION |sdk.ImageAnalysisFeature.TEXT)
    analysis_options.language = self.language
    analysis_options.gender_neutral_caption = True
    image_analyzer = sdk.ImageAnalyzer(self.service_options, vision_source, analysis_options)
    result = image_analyzer.analyze()
    return result
  def do_preprocessing(self):
    result = self.do_analysis()
    data_dict = {}
    line_dict = {}
    if result.reason == sdk.ImageAnalysisResultReason.ANALYZED:
        if result.caption is not None:
          data_dict['Caption'] = {result.caption.content:result.caption.confidence}
        if result.text is not None:
            for line in result.text.lines:
                for word in line.words:
                  line_dict[line.content] = {word.content:word.confidence}
        return data_dict, line_dict
    else:
        error_details = sdk.ImageAnalysisErrorDetails.from_result(result)
        print(" Analysis failed.")
        print("   Error reason: {}".format(error_details.reason))
        print("   Error code: {}".format(error_details.error_code))
        print("   Error message: {}".format(error_details.message))
        return error_details, _

In [ ]:
instance = do_classification(url=url,service_options=sdk.VisionServiceOptions(VISION_ENDPOINT,VISION_KEY)).do_preprocessing()
data_dict, line_dict = instance
sentence = []
for key, value in line_dict.items():
  sentence.append(key)

complete_sentence = ' '.join(sentence)

In [ ]:
# Output
complete_sentence
data_dict

Reference:
1. [Use Document Intelligence models](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/how-to-guides/use-sdk-rest-api?view=doc-intel-4.0.0&tabs=linux&pivots=programming-language-python)
2. [Quickstart: Image Analysis 4.0](https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/quickstarts-sdk/image-analysis-client-library-40?tabs=visual-studio%2Clinux&pivots=programming-language-python)